# AWS SDK for Kotlin - Bedrock Converse API Example
This notebook uses [AWS SDK for Kotlin](https://github.com/awslabs/aws-sdk-kotlin/) to implement the basic Bedrock Converse API operations
- `converse`
- `converseStream`
- `converse` with `toolConfig` implementation

Bedrock Converse API is AWS's service that provides a unified interface to interact with various AI language models for building conversational applications.

More examples can be found on [official sample repository](https://github.com/awsdocs/aws-doc-sdk-examples/tree/main/kotlin).
Free feel to contribute to add more use cases.

## Install dependencies

In [1]:
// load version variables
%use @file[resources/version.json](currentDir=".")
%use dataframe

In [33]:
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("aws.sdk.kotlin:bedrockruntime-jvm:$awsSdkVersion")
    }
    import(
        "aws.sdk.kotlin.services.bedrockruntime.*",
        "kotlinx.coroutines.runBlocking"
    )
}

// list the library, if the library is not exist, restart kernel
notebook.currentClasspath.joinToString("\n")

/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-363/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-stdlib-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-363/kotlin-jupyter-script-classpath-shadowed-zip_extracted/lib-0.12.0-363.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-363/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlin-reflect-1.9.23.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-363/kotlin-jupyter-script-classpath-shadowed-zip_extracted/kotlinx-serialization-core-jvm-1.6.3.jar
/Users/gaplo917/Library/Caches/JetBrains/IntelliJIdea2024.3/kotlinNotebook/kotlin-notebook-standalone.eb20de96/kernels/0.12.0-363/kotlin-jupyter-scri

## Load AWS Access Key
By default, there is a [credential chain provider](https://docs.aws.amazon.com/sdk-for-kotlin/latest/developer-guide/credential-providers.html) to look up your aws credentials from your environment.

⚠️ To remain portable and easier to understand, this example use ***custom credential provider*** that create `kotlinNotebookCredentialProvider` in `resources/aws.secret.json`

In [34]:
// Load aws.secret.json variables
%use @file[resources/aws.secret.json](currentDir=".")

In [35]:
val brRuntime = BedrockRuntimeClient.builder().build().withConfig {
    region = "us-east-1"
    credentialsProvider = kotlinNotebookCredentialProvider
}

## Inference via Bedrock Converse API
Use Bedrock Converse API to inference Nova lite model.

In [36]:
import aws.sdk.kotlin.services.bedrockruntime.*
import aws.sdk.kotlin.services.bedrockruntime.model.ContentBlock
import aws.sdk.kotlin.services.bedrockruntime.model.ConversationRole
import aws.sdk.kotlin.services.bedrockruntime.model.InferenceConfiguration
import aws.sdk.kotlin.services.bedrockruntime.model.Message
import aws.sdk.kotlin.services.bedrockruntime.model.SystemContentBlock


val response = runBlocking {
    brRuntime.converse {
        inferenceConfig = InferenceConfiguration {
            maxTokens = 100
            temperature = 0f
        }
        modelId = "amazon.nova-lite-v1:0"
        system = listOf(SystemContentBlock.Text("You are Peter, a helpful AI assistant."))
        messages = listOf(
            Message {
                content = listOf(ContentBlock.Text("What is your name and what is your purpose of existence?"))
                role = ConversationRole.User
            }
        )
    }
}

print(response)

ConverseResponse(additionalModelResponseFields=null,metrics=ConverseMetrics(latencyMs=661),output=Message(value=Message(content=[Text(value=Hello! My name is Peter, and I'm here to assist you. My purpose is to provide helpful information, answer your questions, and support you in various tasks you might need help with, whether it's finding information, offering advice, or just having a conversation. How can I assist you today?)],role=Assistant)),performanceConfig=null,stopReason=EndTurn,trace=null,usage=TokenUsage(inputTokens=21,outputTokens=63,totalTokens=84))

##  Inference via Bedrock Converse Stream
Use Bedrock converse stream API to stream Nova lite model response.
Illustrated how to aggregate the streaming response into final result.

In [9]:
import aws.sdk.kotlin.services.bedrockruntime.model.ConverseStreamRequest
import kotlinx.coroutines.flow.fold
import kotlinx.coroutines.flow.reduce

runBlocking {
    val result = brRuntime.converseStream(ConverseStreamRequest {
        inferenceConfig = InferenceConfiguration {
            maxTokens = 100
            temperature = 0f
        }
        modelId = "amazon.nova-lite-v1:0"
        system = listOf(SystemContentBlock.Text("You are Peter, a helpful AI assistant."))
        messages = listOf(
            Message {
                content = listOf(ContentBlock.Text("What is your name and what is your purpose of existence?"))
                role = ConversationRole.User
            }
        )
    }) {
        it.stream?.fold("") { acc, e ->
            println(e)
            acc + (e.asContentBlockDeltaOrNull()?.delta?.asTextOrNull() ?: "")
        }
    }

    println("""
    === Aggregated delta text ===
    $result
    """.trimIndent())
}

MessageStart(value=MessageStartEvent(role=Assistant))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value=Hello)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value=!)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value= My)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value= name)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value= is)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value= Peter)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value=,)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value= and)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value= I)))
ContentBlockDelta(value=ContentBlockDeltaEvent(contentBlockIndex=0,delta=Text(value=')))
ContentBlockDelta(value=ContentBloc

## Inference with Tool use
Inference with tool use (functional call).

1. Process the converse ToolUse response
2. Trigger mock getWeatherAPI
3. Continue with ToolUseResult

In [37]:
import aws.sdk.kotlin.services.bedrockruntime.model.*
import aws.smithy.kotlin.runtime.content.Document
import aws.smithy.kotlin.runtime.content.buildDocument

/**
 * Mock API to get weather information
 */
fun getWeatherAPI(date: String, city: String, country: String): Document {
    println("=== calling weather API with date=$date, city=$city, country=$country ===")
    return buildDocument {
        "date" to date
        "city" to city
        "country" to country
        "unit" to "celsius"
        "tempature" to "24"
    }.also {
        println(it)
    }
}

runBlocking {
    var stopReason: StopReason?
    val conversations: MutableList<Message> = mutableListOf()
    do {
        println("=== calling bedrock ===")
        val response = brRuntime.converse {
            inferenceConfig = InferenceConfiguration {
                maxTokens = 1000
                temperature = 0f
            }
            modelId = "amazon.nova-lite-v1:0"
            messages = listOf(
                Message {
                    content = listOf(
                        ContentBlock.Text("What's today wether in Seattle on 2025-01-01?"),
                    )
                    role = ConversationRole.User
                },
                *conversations.toTypedArray()
            )
            toolConfig {
                tools = listOf(Tool.ToolSpec(ToolSpecification {
                    description = "Get weather tool with city, country, and date"
                    name = "GetWeatherTool"
                    inputSchema = ToolInputSchema.Json(
                        buildDocument {
                            "type" to "object"
                            "properties" to buildDocument {
                                "city" to buildDocument {
                                    "type" to "string"
                                    "description" to "City name"
                                    "example" to "San Francisco"
                                }
                                "country" to buildDocument {
                                    "type" to "string"
                                    "description" to "Country code (ISO 3166-1 alpha-2)"
                                    "example" to "US"
                                }
                                "date" to buildDocument {
                                    "type" to "string"
                                    "format" to "date"
                                    "description" to "Date for weather forecast (YYYY-MM-DD)"
                                    "example" to "2024-01-15"
                                }
                            }
                            "required" to buildList {
                                add("city")
                                add("country")
                                add("date")
                            }
                        })
                }))
            }
        }
        println(response)

        stopReason = response.stopReason
        val message = response.output?.asMessageOrNull()
        val toolUseContentBlock = message?.content?.find {
            it is ContentBlock.ToolUse
        }

        if(toolUseContentBlock != null) {
            conversations.add(message)

            val toolUse = toolUseContentBlock.asToolUse()
            val toolUseMap =  toolUse.input?.asMap()
            val date = toolUseMap?.get("date")?.asStringOrNull()
            val city = toolUseMap?.get("city")?.asStringOrNull()
            val country = toolUseMap?.get("country")?.asStringOrNull()
            val _toolUseId = toolUse.toolUseId

            println("=== found ToolUse: $toolUse ===")
            if(date != null && city != null && country != null) {
                ContentBlock.ToolResult(ToolResultBlock {
                    content = listOf(ToolResultContentBlock.Json(
                        getWeatherAPI(date, city, country)
                    ))
                    status = ToolResultStatus.Success
                    toolUseId = _toolUseId
                }).also {
                    conversations.add(Message {
                        content = listOf(it)
                        role = ConversationRole.User
                    })
                }
            }
        }
    } while (stopReason != StopReason.EndTurn)
}

=== calling bedrock ===
ConverseResponse(additionalModelResponseFields=null,metrics=ConverseMetrics(latencyMs=862),output=Message(value=Message(content=[Text(value=<thinking>The User has asked for the weather in Seattle on 2025-01-01. I need to use the GetWeatherTool to retrieve this information. I will call the tool with the provided date, city, and country.</thinking>
), ToolUse(value=ToolUseBlock(input={"date":"2025-01-01","country":"US","city":"Seattle"},name=GetWeatherTool,toolUseId=tooluse_Ng_H-fcQQiG9LyqXtQCU9w))],role=Assistant)),performanceConfig=null,stopReason=ToolUse,trace=null,usage=TokenUsage(inputTokens=518,outputTokens=154,totalTokens=672))
=== found ToolUse: ToolUseBlock(input={"date":"2025-01-01","country":"US","city":"Seattle"},name=GetWeatherTool,toolUseId=tooluse_Ng_H-fcQQiG9LyqXtQCU9w) ===
=== calling weather API with date=2025-01-01, city=Seattle, country=US ===
{"date":"2025-01-01","city":"Seattle","country":"US","unit":"celsius","tempature":"24"}
=== calling be